# Gradient Blow-up in PINN Simulations for rising-bubble Systems with Moving Boundaries

***Focused on the torch.Tensor.backward()***
    
**Woojeong Kim** *8/7/2024*

    
#### Introduction
The simulation of two-phase fluid systems using the Navier-Stokes-Cahn-Hilliard (NSCH) equations presents significant challenges, particularly when dealing with moving boundaries. One prominent example is the 2-dimensional rising-bubble system in two fluids. In this PDE system simulation, the density discontinuity at the moving boundary can cause **blow-up gradients problem** in the neural network forward pass, complicating the training process of Physics-Informed Neural Networks (PINNs). This writing explores the difficulties encountered in such simulations and proposes a method to overcome these challenges using `Normalize initial input variable`, `Normalization on a layer`, `Adjust domain range for the loss computation`,`gradient clipping` and `adaptive loss weighting`.

#### Blow-up gradient phenominon in PINN Simulation of NSCH Systems
The primary difficulty in simulating NSCH systems with moving boundaries arises from the density discontinuity at the interface of two fluids. On the 2-dimensional domain, density is our goal to expect as simulation result as a variable in neural network forward pass. The discontinuity of this variable leads to gradients that can blow up during the neural network forward pass since two different values in small region brings enormous increasing gradient theoretically. Specifically, the density variable, \(\rho\), exhibits sharp changes at the moving boundary of bubble border line becuase the rising-bubble is located between two different fluids with different density for each. 

This rapid change of density causes the gradient of the density variable to become excessively large. When the loss function includes terms dependent on these gradients, the resulting gradients during backpropagation can become unmanageable, leading to errors in loss computation and training instability.

#### Graph Recording for Gradient Computation
During the backward pass, the gradients of the neural network parameters are computed using automatic differentiation. This process involves constructing a computational graph during the forward pass, which records the operations performed on the inputs. When the `.backward()` command is called, the chain rule is applied to this graph to compute the gradients of the loss with respect to each variable.

#### Issues with Blow-Up Gradients
When gradients blow up due to the density discontinuity, the loss term can become infinite or undefined, causing errors in the backward pass. This issue makes it impossible to compute meaningful gradients, halting the training process. Consequently, the PINN model fails to complete on learning an accurate solution to the NSCH equations.

#### Gradient Clipping as a Solution
To address the blow-up gradient problem, we implement gradient clipping between the loss computation and the backward operation. Gradient clipping involves setting a threshold beyond which gradients are scaled down to a manageable size. This technique ensures that no gradient exceeds a predefined maximum value, preventing the loss term from becoming infinite.

```python
# Example code for gradient clipping in PyTorch
import torch

# Assuming loss is computed
loss = compute_loss()

# Perform gradient clipping
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

# Backward pass
loss.backward()
```

#### Adjusting Coefficients for Fluid Interfaces
By bounding the gradient of the density variable in the forward pass, we can adaptively adjust the coefficients for loss terms on different regions in the domain. Specifically, we assign different weights to the loss terms for the interior of each fluid and the moving boundary interface.

- **Interior of Each Fluid**: In these regions, the collocation points should primarily focus on minimizing the Navier-Stokes PDE (NSpde) loss term. To mitigate the effect of large density gradients on the boundary of each fluid, we use an adaptive coefficient:
  $$
  \text{coefficient}_{\text{NSpde}} = \frac{1}{1 + 0.1 \cdot \nabla \rho}
  $$
  This coefficient reduces the influence of high gradients, stabilizing the loss term.

- **Moving Boundary Interface**: At the interface of moving boundary of bubble, the primary concern is the surface tension force and the phase field's accuracy. We assign a different adaptive coefficient to these terms:
  $$
  \text{coefficient}_{\text{boundary}} = \frac{\text{difference of density in small region centered at each training point}}{0.00001 + \nabla \rho}
  $$
  This coefficient emphasizes the importance of accurately recognizing only the interface(Moving boundary of rising bubble) dynamics while preventing blow-up gradients. This helps us to differentiate the boundary training points and non-boundary training points through 2-dimensional domain for training collocation points since difference of density of the non-boundary training collocation points is zero and the one of the boundary training collocation points is non-zero. After this recognizing only the collocation points on boundary collocation points by giving nonzero coefficient as weight of loss term, the denominator \( 0.00001 + \nabla \rho \) is trainined to be small as a part of the process for decreasing this loss term while neural network training.

#### Implementation Strategy
To implement this strategy, we modify the loss function to include these adaptive coefficients. The loss function for the PINN model is thus a weighted sum of the NSpde loss term and the surface tension force loss term, with the weights dynamically adjusted based on the gradient of the density variable.

```python
# Example code for adaptive loss weighting
def compute_adaptive_loss(u, v, rho, surface_tension_error):
    grad_rho = torch.autograd.grad(rho.sum(), inputs, create_graph=True)[0]
    coeff_NSpde = 1 / (1 + 0.1 * grad_rho)
    coeff_boundary = (rho_diff_small_region) / (0.00001 + grad_rho)
    
    loss_NSpde = compute_NSpde_loss(u, v, rho) * coeff_NSpde
    loss_boundary = compute_boundary_loss(surface_tension_error) * coeff_boundary
    
    total_loss = loss_NSpde + loss_boundary
    return total_loss
```

#### Conclusion
By employing gradient clipping and adaptive loss weighting, we can effectively manage the challenges posed by density discontinuities in NSCH systems with moving boundaries. This approach allows the PINN model to maintain stable training, accurately capture the dynamics of two-phase fluids, and reduce loss errors. These techniques, grounded in the principles of automatic differentiation and gradient management, offer a robust solution for simulating complex fluid systems with moving boundaries.